In [1]:
# default_exp rank

In [89]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [5]:
# export
import os
import sys
sys.path.append('../mylib/')
from code.eda import * 
from code.config import args
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from loguru import logger
from utils.pickle import PickleWrapper
import scipy.sparse as sp

In [91]:
# export
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

In [92]:
args

Namespace(DATA_DIR='../../data/contest/kdd_debiasing', mode='train', now_phase=9, submit_fp='/Users/luoyonggui/Downloads/baseline1_itemcf4.csv', time_decay=0.875, topk=500, window=10)

# val

In [5]:
# export
def val1(df, topk, click_val):
    """
    """
    sr = df.groupby('user_id')['item_id_pred'].agg(list)
    r_list = []
    pred_num = click_val.shape[0]
    for line in click_val.itertuples():
        try:
            df.loc[df.user_id==line.user_id, 'item_id_true'] = line.item_id
            t = sr[line.user_id]
            i = t.index(line.item_id)
        except:
            i = 99999

        r_list.append(i)
    r_series = pd.Series(r_list)
    for i in range(25, topk+1, 25):
        logger.info(f'recall{i}:{r_series[r_series<i].shape[0]/pred_num}')
        
#     r_series[r_series<9999].plot(kind='box', title='pp', rot=45)
    return df

# load click data

In [93]:
# export
all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)

2020-06-06 09:33:32.447 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 0
2020-06-06 09:33:33.489 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-06-06 09:33:33.490 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 1
2020-06-06 09:33:34.562 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-06-06 09:33:34.564 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 2
2020-06-06 09:33:35.633 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-06-06 09:33:35.635 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 3
2020-06-06 09:33:36.913 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-06-06 09:33:36.914 | INFO     | code.ed

In [7]:
# export
item_feat = get_item_data()

/root/data1/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [76]:
item_feat.iloc[:, 1:129].shape

(108916, 128)

In [209]:
item_feat.iloc[:, 129:257].shape

(108916, 128)

In [8]:
# export
item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

item_feat.set_index('item_id', inplace=True)

In [9]:
# export
phase = [0, 1, 2, 3, 5, 6, 4]

# phase = [4]
# match_num = 200
match_num = 500

In [185]:
# 计算每个phase的item_id user_cnt  # 貌似每个phase的item只会被user唯一点击一次，那就没必要统计了。。。
def t(s):
    return s.nunique()
all_click_df[0][0].groupby('item_id')['user_id'].agg(t)

item_id
1          4
1000      13
100000     8
10001      5
100013     5
          ..
99990      6
99991      3
99993      5
99994      5
99998      5
Name: user_id, Length: 40775, dtype: int64

In [177]:
# temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
temp_ = all_click_df[0][0].groupby('item_id')['user_id'].agg(list)

In [179]:
temp_

item_id
1                               [1489, 10088, 13420, 19312]
1000      [14546, 9531, 32114, 6934, 20558, 30360, 33492...
100000    [20800, 17064, 33901, 5699, 8541, 1940, 20705,...
10001                      [6572, 17178, 25162, 6383, 8264]
100013                    [18924, 32480, 7696, 6319, 32706]
                                ...                        
99990              [30399, 8084, 2969, 21225, 32152, 34183]
99991                                  [9430, 32852, 31855]
99993                   [23622, 22023, 12546, 31661, 21784]
99994                     [9287, 6305, 28964, 17648, 22420]
99998                    [24217, 9796, 20948, 25551, 16599]
Name: user_id, Length: 40775, dtype: object

In [178]:
temp_.str.len().describe()

count    40775.000000
mean         6.409246
std          6.358793
min          1.000000
25%          4.000000
50%          5.000000
75%          7.000000
max        234.000000
Name: user_id, dtype: float64

In [ ]:
for p in phase:
    # 获取test用户的近n次点击
    temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
    temp_ = pd.DataFrame(temp_)
    for i in range(5):
        temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        
    # 计算每个phase的item_id cnt
    all_click_df[0][0].groupby('item_id')['user_id'].count()

# load match result

In [125]:
# export
def load_match_items():
    r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527.pkl'))

    r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527.pkl'))
    
    r_itemcf_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527_phase4.pkl'))

    r_binn_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527_phase4.pkl'))

    
    r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_0527.pkl'))
    r_list_w2v = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_w2v_0602_all.pkl'))
    r_itemcf = r_itemcf + r_itemcf_phase4
    r_binn = r_binn + r_binn_phase4
    r_itemcf_yl = [r_itemcf_yl[0], r_itemcf_yl[1],r_itemcf_yl[2],r_itemcf_yl[3],r_itemcf_yl[5],r_itemcf_yl[6],r_itemcf_yl[4]]
    
    r_list_w2v = [r_list_w2v[0], r_list_w2v[1],r_list_w2v[2],r_list_w2v[3],r_list_w2v[5],r_list_w2v[6],r_list_w2v[4]]
    return r_itemcf, r_binn, r_itemcf_yl, r_list_w2v


def load_match_items789():
    r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0605_789.pkl'))

    r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0605_789.pkl'))
    
    r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_0606_789.pkl'))
    r_list_w2v = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_w2v_0605_789.pkl'))

    return r_itemcf, r_binn, r_itemcf_yl, r_list_w2v


In [126]:
r_itemcf, r_binn, r_itemcf_yl, r_list_w2v = load_match_items789()

In [127]:
phase

[7, 8, 9]

In [128]:
# export

def data_preporcess(recall_list, match_num, phase, mode='train'):
    
    itemcf, r_binn, r_itemcf_yl, r_list_w2v = recall_list
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in range(len(phase)):
        r_itemcf[i]['phase'] = phase[i]
        r_binn[i]['phase'] = phase[i]
        r_itemcf_yl[i]['phase'] = phase[i]
        r_list_w2v[i]['phase'] = phase[i]
        df1 = df1.append(r_itemcf[i])
        df2 = df2.append(r_binn[i])
        df3 = df3.append(r_itemcf_yl[i])
        df4 = df4.append(r_list_w2v[i])

    df1_ = df1[df1['rank']<(match_num+1)]
    df2_ = df2[df2['rank']<(match_num+1)]
    df3_ = df3[df3['rank']<(match_num+1)]
    df4_ = df4[df4['rank']<(match_num+1)]

    print('merge Multi-channel recall...')
    cols = 'user_id item_id_pred phase'.split()
    
    if mode == 'train':
        cols = 'user_id item_id_pred item_id_true phase'.split()
    df = pd.merge(pd.merge(pd.merge(df1_, 
                                    df2_, on=cols, how='outer'), 
                                      df3_, on=cols, how='outer'), 
                                      df4_, on=cols, how='outer')

    df = df.fillna(0)
    if mode == 'train':
        df['label'] = (df['item_id_pred'] == df['item_id_true']).map(int)


    dft = pd.DataFrame()
    for p in tqdm(phase):
        # 获取test用户的近n次点击
        temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
        temp_ = pd.DataFrame(temp_)
        for i in range(0, 3):
            temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        dft = dft.append(temp_.reset_index())
        # 计算每个phase的item_id cnt
        t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
        df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

    df = pd.merge(df, dft.drop(columns='item_id'))

    

    # 太大了，不建议存储    
    # df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))

    # df = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))
    return df

In [129]:
recall_list = [r_itemcf, r_binn, r_itemcf_yl, r_list_w2v]
df = data_preporcess(recall_list, match_num, phase)

merge Multi-channel recall...


100%|██████████| 3/3 [03:08<00:00, 62.73s/it]


In [130]:
df.columns

Index(['user_id', 'item_id_pred', 'score_x', 'rank_x', 'item_id_true', 'phase',
       'score_y', 'rank_y', 'score_x', 'rank_x', 'score_y', 'rank_y', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3'],
      dtype='object')

In [132]:
df.columns = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3']

In [133]:
df.head()

,user_id,item_id_pred,score_1,rank_1,item_id_true,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,label,item_cnt,last_1,last_2,last_3
0,7,79771,85.970305,1.0,117546,7,15.617781,3.0,0.413184,1.0,132.537654,3.0,0,115.0,44390,80626,12845
1,7,113031,70.487979,2.0,117546,7,15.572158,4.0,0.374192,2.0,122.875506,8.0,0,174.0,44390,80626,12845
2,7,57786,62.176548,3.0,117546,7,10.107695,14.0,0.366759,3.0,117.948432,17.0,0,104.0,44390,80626,12845
3,7,114900,61.809225,4.0,117546,7,16.573303,2.0,0.057253,40.0,118.777295,15.0,0,139.0,44390,80626,12845
4,7,5577,58.101950,5.0,117546,7,17.074922,1.0,0.016208,324.0,113.328205,22.0,0,138.0,44390,80626,12845


In [17]:
# temp_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features.pkl'))

# temp4_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features_phase4.pkl'))

In [134]:
df.shape

(6612360, 17)

In [137]:
def t1(s):
    return df.item_id_pred.map(lambda x: item_feat[f'{s}_vec'][x] if x in item_feat[f'{s}_vec'] else np.zeros(128))

In [139]:
pool = Pool(2)
r1 = pool.map(t1, 'text img'.split())

In [140]:
df['item_id_pred_text_vec'] = r1[0]
df['item_id_pred_img_vec'] = r1[1]

In [141]:


def t2(c):
#     dff[f'{c}_text_vec']

    c, a = c.split(',')
    print(c, a)
    return df[c].map(lambda x: item_feat[f'{a}_vec'][x] if x in item_feat[f'{a}_vec'] else np.zeros(128))


In [142]:
from multiprocessing.dummy import Pool
pool = Pool(6)

r = pool.map(t2, 'last_1,text last_2,text last_3,text last_1,img last_2,img last_3,img'.split())

last_1 text
last_2last_3last_1 img
 text
last_2 img
last_3 img
 text


In [143]:
df['last1_text_vec'] = r[0]

df['last1_img_vec'] = r[3]

df['last2_text_vec'] = r[1]
df['last2_img_vec'] = r[4]

df['last3_text_vec'] = r[2]
df['last3_img_vec'] = r[5]

In [144]:
# from multiprocessing.dummy import Pool
# pool = Pool(6)
# r = pool.map(t3, 'last1,text last2,text last3,text last1,img last2,img last3,img'.split())
logger.info('sim1_text')
df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last1_text_vec']).map(sum)
logger.info('sim1_img')
df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last1_img_vec']).map(sum)
logger.info('sim2_text')
df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last2_text_vec']).map(sum)
logger.info('sim2_img')
df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last2_img_vec']).map(sum)
logger.info('sim3_text')
df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last3_text_vec']).map(sum)
logger.info('sim3_img')
df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last3_img_vec']).map(sum)

2020-06-07 09:58:49.871 | INFO     | __main__:<module>:4 - sim1_text
2020-06-07 10:02:52.204 | INFO     | __main__:<module>:6 - sim1_img
2020-06-07 10:06:47.904 | INFO     | __main__:<module>:8 - sim2_text
2020-06-07 10:10:58.761 | INFO     | __main__:<module>:10 - sim2_img
2020-06-07 10:15:04.090 | INFO     | __main__:<module>:12 - sim3_text
2020-06-07 10:19:04.698 | INFO     | __main__:<module>:14 - sim3_img


In [146]:
df.shape

(6612360, 31)

In [53]:
# df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_features_phase4.pkl'))

In [27]:
# sim_text_dict = {}
# sim_img_dict = {}

# FE

In [151]:
# export
def fe(df):
    for i in tqdm('rank score'.split()):
        df[f'{i}1_sub_{i}2'] = df[f'{i}_1']-df[f'{i}_2']
        df[f'{i}1_add_{i}2'] = df[f'{i}_1']+df[f'{i}_2']
        df[f'{i}1_mul_{i}2'] = df[f'{i}_1']*df[f'{i}_2']

        df[f'{i}1_sub_{i}3'] = df[f'{i}_1']-df[f'{i}_3']
        df[f'{i}1_add_{i}3'] = df[f'{i}_1']+df[f'{i}_3']
        df[f'{i}1_mul_{i}3'] = df[f'{i}_1']*df[f'{i}_3']

        df[f'{i}1_sub_{i}4'] = df[f'{i}_1']-df[f'{i}_4']
        df[f'{i}1_add_{i}4'] = df[f'{i}_1']+df[f'{i}_4']
        df[f'{i}1_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_4']

        df[f'{i}2_sub_{i}3'] = df[f'{i}_2']-df[f'{i}_3']
        df[f'{i}2_add_{i}3'] = df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}2_mul_{i}3'] = df[f'{i}_2']*df[f'{i}_3']

        df[f'{i}2_sub_{i}4'] = df[f'{i}_2']-df[f'{i}_4']
        df[f'{i}2_add_{i}4'] = df[f'{i}_2']+df[f'{i}_4']
        df[f'{i}2_mul_{i}4'] = df[f'{i}_2']*df[f'{i}_4']

        df[f'{i}3_sub_{i}4'] = df[f'{i}_3']-df[f'{i}_4']
        df[f'{i}3_add_{i}4'] = df[f'{i}_3']+df[f'{i}_4']
        df[f'{i}3_mul_{i}4'] = df[f'{i}_3']*df[f'{i}_4']
        
        df[f'{i}1_add_{i}2_add_{i}3'] = df[f'{i}_1']+df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}1_mul_{i}2_mul_{i}3'] = df[f'{i}_1']*df[f'{i}_2']*df[f'{i}_3']
        
        df[f'{i}1_add_{i}2_add_{i}4'] = df[f'{i}_1']+df[f'{i}_2']+df[f'{i}_4']
        df[f'{i}1_mul_{i}2_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_2']*df[f'{i}_4']
        
        df[f'{i}4_add_{i}2_add_{i}3'] = df[f'{i}_4']+df[f'{i}_2']+df[f'{i}_3']
        df[f'{i}4_mul_{i}2_mul_{i}3'] = df[f'{i}_4']*df[f'{i}_2']*df[f'{i}_3']
        
        df[f'{i}1_add_{i}2_add_{i}3_add_{i}4'] = df[f'{i}_1']+df[f'{i}_2']+df[f'{i}_3']+df[f'{i}_4']
        df[f'{i}1_mul_{i}2_mul_{i}3_mul_{i}4'] = df[f'{i}_1']*df[f'{i}_2']*df[f'{i}_3']*df[f'{i}_4']
        
#     df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last_1_text_vec']).map(sum)

#     df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last_1_img_vec']).map(sum)

#     df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last_2_text_vec']).map(sum)
#     df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last_2_img_vec']).map(sum)

#     df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last_3_text_vec']).map(sum)
#     df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last_3_img_vec']).map(sum)

#     df['sim4_text'] = (df['item_id_pred_text_vec'] * df['last_4_text_vec']).map(sum)
#     df['sim4_img'] = (df['item_id_pred_img_vec'] * df['last_4_img_vec']).map(sum)

#     df['sim5_text'] = (df['item_id_pred_text_vec'] * df['last_5_text_vec']).map(sum)
#     df['sim5_img'] = (df['item_id_pred_img_vec'] * df['last_5_img_vec']).map(sum)
    
    

    df['sim1_text_img'] = df['sim1_text'] *  df['sim1_img']
    df['sim2_text_img'] = df['sim2_text'] *  df['sim2_img']
    df['sim3_text_img'] = df['sim3_text'] *  df['sim3_img']
#     df['sim4_text_img'] = df['sim4_text'] *  df['sim4_img']
#     df['sim5_text_img'] = df['sim5_text'] *  df['sim5_img']

    df['sim12_text'] = df['sim1_text'] + df['sim2_text']
    df['sim123_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text']
#     df['sim1234_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text']
#     df['sim12345_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text'] + df['sim5_text']

    df['sim12_img'] = df['sim1_img'] + df['sim2_img']
    df['sim123_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img']
#     df['sim1234_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img']
#     df['sim12345_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img'] + df['sim5_img']

    df['sim12_text_img'] = df['sim1_text_img'] + df['sim2_text_img']
    df['sim123_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img']
#     df['sim1234_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img']
#     df['sim12345_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img'] + df['sim5_text_img']
 
    
    return df

In [152]:
df = fe(df)

100%|██████████| 2/2 [00:00<00:00,  2.26it/s]


In [153]:
df.shape

(6612360, 92)

In [101]:
dff.shape

(15585787, 69)

In [23]:
dff.label.value_counts()

0    15581708
1        4079
Name: label, dtype: int64

In [102]:
dff.label.value_counts()

0    15581708
1        4079
Name: label, dtype: int64

In [103]:
dff.user_id.nunique()

12080

In [24]:
cols = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img',
       'rank1_add_rank2_add_rank3', 'rank1_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank4', 'rank1_mul_rank2_mul_rank4',
       'rank4_add_rank2_add_rank3', 'rank4_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank3_add_rank4',
       'rank1_mul_rank2_mul_rank3_mul_rank4', 'score1_add_score2_add_score3',
       'score1_mul_score2_mul_score3', 'score1_add_score2_add_score4',
       'score1_mul_score2_mul_score4', 'score4_add_score2_add_score3',
       'score4_mul_score2_mul_score3',
       'score1_add_score2_add_score3_add_score4',
       'score1_mul_score2_mul_score3_mul_score4']

In [68]:
dff[['sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img']].head(5)

,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img
786,127.659306,167.783107,357.839126,324.319944,208.024795,143.955396,21419.075115,116054.365359,29946.291778,485.498432,693.523227,492.103052,636.058448,137473.440474,167419.732252
787,159.745719,56.719114,306.328177,72.745689,113.429797,73.263342,9060.635599,22284.054405,8310.245996,466.073897,579.503694,129.464803,202.728145,31344.690004,39654.936000
788,344.369137,130.674165,119.010146,57.891801,97.820684,56.114521,45000.149376,6889.711704,5489.160804,463.379282,561.199966,188.565966,244.680487,51889.861080,57379.021884
789,220.072454,220.798732,331.988678,110.750306,233.445790,257.246191,48591.718918,36767.847665,60053.040254,552.061132,785.506922,331.549038,588.795229,85359.566583,145412.606837
790,120.077336,167.040284,390.074651,144.685020,54.171616,82.703507,20057.752274,56437.958720,4480.182574,510.151987,564.323603,311.725304,394.428811,76495.710993,80975.893568


In [67]:
dff = fe(dff)

In [156]:
cols = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img',
       'rank1_add_rank2_add_rank3', 'rank1_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank4', 'rank1_mul_rank2_mul_rank4',
       'rank4_add_rank2_add_rank3', 'rank4_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank3_add_rank4',
       'rank1_mul_rank2_mul_rank3_mul_rank4', 'score1_add_score2_add_score3',
       'score1_mul_score2_mul_score3', 'score1_add_score2_add_score4',
       'score1_mul_score2_mul_score4', 'score4_add_score2_add_score3',
       'score4_mul_score2_mul_score3',
       'score1_add_score2_add_score3_add_score4',
       'score1_mul_score2_mul_score3_mul_score4']

In [157]:
# df[cols].to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_train_features_789.pkl'))

In [159]:
df.shape

(6612360, 92)

In [11]:
dff = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_train_features_new.pkl'))

In [161]:
dfa = dff[cols].append(df[cols])

In [162]:
dfa.shape

(22198147, 84)

In [16]:
# add cocur features
#export
def load_co_occurance_matrix(save_dir,penalty1,penalty2,penalty3):
    id2item = pd.read_pickle(os.path.join(save_dir, f'id2item_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
    item2id = pd.read_pickle(os.path.join(save_dir, f'item2id_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
    co_occurance_matrix = sp.load_npz(os.path.join(save_dir, f'item_item_matrix_{penalty1}_{penalty2}_{penalty3}.npz'))
    return co_occurance_matrix, id2item, item2id

In [23]:
window = 20
mode = 'train'
train_data_matrixs, id2items, item2ids = {}, {}, {}
for p in phase:
    save_dir=f'{os.path.join(args.DATA_DIR, "data_gen/matrix_w")}{window}_{mode}_{p}'
    train_data_matrix, id2item, item2id = load_co_occurance_matrix(save_dir=save_dir, 
                                                              penalty1=True, penalty2=True,
                                                              penalty3=0.9)
    train_data_matrixs[p] = train_data_matrix
    id2items[p] = id2item
    item2ids[p] = item2id

In [73]:
train_data_matrixs[0]

<40775x40775 sparse matrix of type '<class 'numpy.float64'>'
	with 4539818 stored elements in Compressed Sparse Column format>

In [66]:
item2ids[2]['63972']

32221

In [ ]:
dff[f'item_id_pred_last_1_cc'] = 0
dff[f'last_1_item_id_pred_cc'] = 0

dff[f'item_id_pred_last_2_cc'] = 0
dff[f'last_2_item_id_pred_cc'] = 0

dff[f'item_id_pred_last_3_cc'] = 0
dff[f'last_3_item_id_pred_cc'] = 0

In [61]:
dff.shape

(15585787, 87)

In [62]:
dff.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'item_id_true', 'phase',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'label',
       'item_cnt', 'last_1', 'last_2', 'last_3', 't', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_s

In [30]:
def t(line):
    p = 
    r = train_data_matrixs[line.phase][item2ids[line.phase][line.item_id_pred], item2ids[line.phase][line.last_1]]
dff.loc[:, 'item_id_pred_last_1_cc'] = dff.apply(lambda line: , axis=1)
    

KeyError: ('63972', 'occurred at index 539')

In [ ]:
dff.loc[:, 'item_id_pred_last_1_cc'].iloc[:10]

In [ ]:
dff.loc[:, 'item_id_pred_last_2_cc'] = dff.apply(lambda line: train_data_matrixs[line.phase][item2ids[line.phase][line.item_id_pred], item2ids[line.phase][line.last_2]], axis=1)
    

In [ ]:
dff.loc[:, 'item_id_pred_last_3_cc'] = dff.apply(lambda line: train_data_matrixs[line.phase][item2ids[line.phase][line.item_id_pred], item2ids[line.phase][line.last_3]], axis=1)
    

In [31]:
dff.head(2)

,user_id,item_id_pred,score_1,rank_1,item_id_true,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,label,item_cnt,last_1,last_2,last_3,t,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank1_sub_rank4,rank1_add_rank4,rank1_mul_rank4,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,rank2_sub_rank4,rank2_add_rank4,rank2_mul_rank4,rank3_sub_rank4,rank3_add_rank4,rank3_mul_rank4,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score1_sub_score4,score1_add_score4,score1_mul_score4,score2_sub_score3,score2_add_score3,score2_mul_score3,score2_sub_score4,score2_add_score4,score2_mul_score4,score3_sub_score4,score3_add_score4,score3_mul_score4,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img,rank1_add_rank2_add_rank3,rank1_mul_rank2_mul_rank3,rank1_add_rank2_add_rank4,rank1_mul_rank2_mul_rank4,rank4_add_rank2_add_rank3,rank4_mul_rank2_mul_rank3,rank1_add_rank2_add_rank3_add_rank4,rank1_mul_rank2_mul_rank3_mul_rank4,score1_add_score2_add_score3,score1_mul_score2_mul_score3,score1_add_score2_add_score4,score1_mul_score2_mul_score4,score4_add_score2_add_score3,score4_mul_score2_mul_score3,score1_add_score2_add_score3_add_score4,score1_mul_score2_mul_score3_mul_score4
786,11,66987,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,29.806313,1.0,0,4.0,28210,64689,20317,786,711.084763,557.705309,711.084763,557.705309,711.084763,557.705309,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-29.806313,29.806313,0.0,0.0,0.0,0.0,-29.806313,29.806313,0.0,-29.806313,29.806313,0.0,396575.747305,396575.747305,396575.747305,1422.169526,2133.254289,1115.410618,1673.115926,7.931515e+05,1.189727e+06,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,29.806313,0.0,29.806313,0.0,29.806313,0.0
787,11,94944,0.0,0.0,8052,0,0.0,0.0,0.0,0.0,27.631903,2.0,0,5.0,28210,64689,20317,787,1051.756525,652.120819,1051.756525,652.120819,1051.756525,652.120819,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-27.631903,27.631903,0.0,0.0,0.0,0.0,-27.631903,27.631903,0.0,-27.631903,27.631903,0.0,685872.325816,685872.325816,685872.325816,2103.513049,3155.269574,1304.241637,1956.362456,1.371745e+06,2.057617e+06,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,27.631903,0.0,27.631903,0.0,27.631903,0.0


# model

In [165]:
#export
def get_model_input(df, mode='train'):
    from catboost import Pool, CatBoostClassifier

    cols = ['score_1', 'rank_1',
       'score_2', 'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4',
       'item_cnt', 'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img',
       'rank1_add_rank2_add_rank3', 'rank1_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank4', 'rank1_mul_rank2_mul_rank4',
       'rank4_add_rank2_add_rank3', 'rank4_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank3_add_rank4',
       'rank1_mul_rank2_mul_rank3_mul_rank4', 'score1_add_score2_add_score3',
       'score1_mul_score2_mul_score3', 'score1_add_score2_add_score4',
       'score1_mul_score2_mul_score4', 'score4_add_score2_add_score3',
       'score4_mul_score2_mul_score3',
       'score1_add_score2_add_score3_add_score4',
       'score1_mul_score2_mul_score3_mul_score4']
    df['fscore'] = 0
    df['frank'] = 999
    cat_cols = []
    if mode=='train':
        X_train, X_test, y_train, y_test = df.loc[df['phase']<7, cols], df.loc[df['phase']>6, cols], df.loc[df['phase']<7, 'label'],\
                                               df.loc[df['phase']>6, 'label']                 

        
        train_data = Pool(data=X_train,
                           label=y_train,
                           cat_features=cat_cols)
        val_data = Pool(data=X_test, label=y_test, cat_features=cat_cols) 
        return train_data, val_data, cat_cols
#     cols.remove('label')
    X_test = df[cols]
    test_data = Pool(data=X_test, cat_features=cat_cols) 
    return test_data, cat_cols

In [166]:
train_data, val_data, cat_cols = get_model_input(dfa)

In [168]:
dfa['phase'].value_counts()

6    2481465
5    2432847
4    2293060
8    2278856
9    2271134
3    2204614
2    2179516
1    2149532
7    2062370
0    1844753
Name: phase, dtype: int64

In [198]:
# export
def get_model():
    from catboost import Pool, CatBoostClassifier
    params = {
        'iterations': 20,
        'learning_rate': 0.1,
        'random_seed': 144,
        'custom_metric': 'F1',
        'loss_function': 'Logloss',
        'class_weights': [1, 20],
    #     'use_best_model': False
        }
    print(params)
    model = CatBoostClassifier(**params)
    return model

In [79]:
# val1(r_itemcf[-1], 100, all_click_df[6][1])
# 2020-05-27 17:39:59.717 | INFO     | __main__:val1:24 - recall50:0.15595826468973092
# 2020-05-27 17:39:59.728 | INFO     | __main__:val1:24 - recall100:0.18231740801757276


# fit

In [203]:
model = get_model()

{'iterations': 20, 'learning_rate': 0.1, 'random_seed': 144, 'custom_metric': 'F1', 'loss_function': 'Logloss', 'class_weights': [1, 20]}


In [171]:
# export
# r_itemcf, r_binn, r_itemcf_yl = load_match_items()
# dfa = data_preporcess([r_itemcf, r_binn, r_itemcf_yl], match_num, phase)
# dfa = fe(dfa)
# train_data, val_data, cat_cols = get_model_input(dfa)

r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)

rp = model.predict_proba(val_data)[:, 1]

TypeError: '786        False
787        False
788        False
789        False
790        False
           ...  
9535772    False
9535773    False
9535774    False
9535775    False
9535776    False
Name: phase, Length: 15585787, dtype: bool' is an invalid key

In [186]:
dfa.loc[dfa['phase']==7, 'fscore'] = rp[:dfa[dfa['phase']==7].shape[0]]
dfa.loc[dfa['phase']==7, 'frank'] = dfa[dfa['phase']==7].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(dfa[(dfa['phase']==7)&(dfa['frank']<101)], 100, all_click_df[7][1])

2020-06-07 11:03:36.218 | INFO     | __main__:val1:19 - recall25:0.09738452977184196
2020-06-07 11:03:36.230 | INFO     | __main__:val1:19 - recall50:0.12687813021702837
2020-06-07 11:03:36.232 | INFO     | __main__:val1:19 - recall75:0.1547022815804118
2020-06-07 11:03:36.234 | INFO     | __main__:val1:19 - recall100:0.18141346688925988


In [194]:
dfa.loc[dfa['phase']==8, 'fscore'] = rp[dfa[dfa['phase']==7].shape[0]:(dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])]
dfa.loc[dfa['phase']==8, 'frank'] = dfa[dfa['phase']==8].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(dfa[(dfa['phase']==8)&(dfa['frank']<101)], 100, all_click_df[8][1])

2020-06-07 11:18:26.257 | INFO     | __main__:val1:19 - recall25:0.09130913091309131
2020-06-07 11:18:26.263 | INFO     | __main__:val1:19 - recall50:0.12706270627062707
2020-06-07 11:18:26.266 | INFO     | __main__:val1:19 - recall75:0.15126512651265125
2020-06-07 11:18:26.268 | INFO     | __main__:val1:19 - recall100:0.1782178217821782


In [195]:
ta = (dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])

dfa.loc[dfa['phase']==9, 'fscore'] = rp[ta:(ta+dfa[dfa['phase']==9].shape[0])]
dfa.loc[dfa['phase']==9, 'frank'] = dfa[dfa['phase']==9].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(dfa[(dfa['phase']==9)&(dfa['frank']<101)], 100, all_click_df[9][1])

2020-06-07 11:20:02.694 | INFO     | __main__:val1:19 - recall25:0.0839041095890411
2020-06-07 11:20:02.698 | INFO     | __main__:val1:19 - recall50:0.11815068493150685
2020-06-07 11:20:02.701 | INFO     | __main__:val1:19 - recall75:0.1478310502283105
2020-06-07 11:20:02.704 | INFO     | __main__:val1:19 - recall100:0.1678082191780822


In [ ]:
2020-05-28 17:35:07.937 | INFO     | __main__:val1:24 - recall50:0.17298187808896212
2020-05-28 17:35:07.940 | INFO     | __main__:val1:24 - recall100:0.21087314662273476
                    
2020-06-03 11:54:35.829 | INFO     | __main__:val1:19 - recall50:0.2317408017572762
2020-06-03 11:54:35.835 | INFO     | __main__:val1:19 - recall100:0.3399231191652938
                    
2020-06-03 14:55:11.593 | INFO     | __main__:val1:19 - recall50:0.14332784184514002
2020-06-03 14:55:11.596 | INFO     | __main__:val1:19 - recall100:0.21636463481603516    
                    
2020-06-04 16:59:19.792 | INFO     | __main__:val1:19 - recall25:0.1043382756727073
2020-06-04 16:59:19.795 | INFO     | __main__:val1:19 - recall50:0.15760571114772104
2020-06-04 16:59:19.797 | INFO     | __main__:val1:19 - recall75:0.18451400329489293
2020-06-04 16:59:19.798 | INFO     | __main__:val1:19 - recall100:0.2026359143327842
                    
2020-06-05 12:17:49.300 | INFO     | __main__:val1:19 - recall25:0.10818231740801758
2020-06-05 12:17:49.303 | INFO     | __main__:val1:19 - recall50:0.16254805052169138
2020-06-05 12:17:49.305 | INFO     | __main__:val1:19 - recall75:0.19330038440417352
2020-06-05 12:17:49.306 | INFO     | __main__:val1:19 - recall100:0.21306974190005493                    

In [204]:
cnt = 7
for a in range(18):
    print(a)
    if a == 0:
        r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
    else:
        r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    
    
    if a in [3, 17, 19]:
        rp = model.predict_proba(val_data)[:, 1]
        model.save_model(os.path.join(args.DATA_DIR, f'data_gen/catboost_model500_16_{cnt}.dump'))
        cnt += 1
        if a == 3:
            dfa.loc[dfa['phase']==7, 'fscore'] = rp[:dfa[dfa['phase']==7].shape[0]]
            dfa.loc[dfa['phase']==7, 'frank'] = dfa[dfa['phase']==7].groupby('user_id')['fscore'].rank(method='first', ascending=False)
            r = val1(dfa[(dfa['phase']==7)&(dfa['frank']<101)], 100, all_click_df[7][1])
        if a == 17:
            dfa.loc[dfa['phase']==8, 'fscore'] = rp[dfa[dfa['phase']==7].shape[0]:(dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])]
            dfa.loc[dfa['phase']==8, 'frank'] = dfa[dfa['phase']==8].groupby('user_id')['fscore'].rank(method='first', ascending=False)
            r = val1(dfa[(dfa['phase']==8)&(dfa['frank']<101)], 100, all_click_df[8][1])
        if a == 19:
            ta = (dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])

            dfa.loc[dfa['phase']==9, 'fscore'] = rp[ta:(ta+dfa[dfa['phase']==9].shape[0])]
            dfa.loc[dfa['phase']==9, 'frank'] = dfa[dfa['phase']==9].groupby('user_id')['fscore'].rank(method='first', ascending=False)
            r = val1(dfa[(dfa['phase']==9)&(dfa['frank']<101)], 100, all_click_df[9][1])

#     dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

#     dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

#     r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0
1
2
3


2020-06-07 18:26:55.171 | INFO     | __main__:val1:19 - recall25:0.10072342793544797
2020-06-07 18:26:55.174 | INFO     | __main__:val1:19 - recall50:0.13800779076238176
2020-06-07 18:26:55.176 | INFO     | __main__:val1:19 - recall75:0.1686143572621035
2020-06-07 18:26:55.178 | INFO     | __main__:val1:19 - recall100:0.1930996104618809


4
5
6
7
8
9
10
11
12
13
14
15
16
17


2020-06-07 18:36:49.954 | INFO     | __main__:val1:19 - recall25:0.10176017601760176
2020-06-07 18:36:49.957 | INFO     | __main__:val1:19 - recall50:0.1408140814081408
2020-06-07 18:36:49.960 | INFO     | __main__:val1:19 - recall75:0.17106710671067107
2020-06-07 18:36:49.961 | INFO     | __main__:val1:19 - recall100:0.19911991199119913


In [196]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    rp = model.predict_proba(val_data)[:, 1]
    
    dfa.loc[dfa['phase']==7, 'fscore'] = rp[:dfa[dfa['phase']==7].shape[0]]
    dfa.loc[dfa['phase']==7, 'frank'] = dfa[dfa['phase']==7].groupby('user_id')['fscore'].rank(method='first', ascending=False)
    r = val1(dfa[(dfa['phase']==7)&(dfa['frank']<101)], 100, all_click_df[7][1])

    dfa.loc[dfa['phase']==8, 'fscore'] = rp[dfa[dfa['phase']==7].shape[0]:(dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])]
    dfa.loc[dfa['phase']==8, 'frank'] = dfa[dfa['phase']==8].groupby('user_id')['fscore'].rank(method='first', ascending=False)
    r = val1(dfa[(dfa['phase']==8)&(dfa['frank']<101)], 100, all_click_df[8][1])

    ta = (dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])

    dfa.loc[dfa['phase']==9, 'fscore'] = rp[ta:(ta+dfa[dfa['phase']==9].shape[0])]
    dfa.loc[dfa['phase']==9, 'frank'] = dfa[dfa['phase']==9].groupby('user_id')['fscore'].rank(method='first', ascending=False)
    r = val1(dfa[(dfa['phase']==9)&(dfa['frank']<101)], 100, all_click_df[9][1])

#     dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

#     dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

#     r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0


2020-06-07 11:24:34.978 | INFO     | __main__:val1:19 - recall25:0.1001669449081803
2020-06-07 11:24:34.982 | INFO     | __main__:val1:19 - recall50:0.13299944351697274
2020-06-07 11:24:34.984 | INFO     | __main__:val1:19 - recall75:0.16583194212576516
2020-06-07 11:24:34.986 | INFO     | __main__:val1:19 - recall100:0.19198664440734559
2020-06-07 11:25:45.380 | INFO     | __main__:val1:19 - recall25:0.09460946094609461
2020-06-07 11:25:45.383 | INFO     | __main__:val1:19 - recall50:0.13366336633663367
2020-06-07 11:25:45.385 | INFO     | __main__:val1:19 - recall75:0.15951595159515952
2020-06-07 11:25:45.387 | INFO     | __main__:val1:19 - recall100:0.1848184818481848
2020-06-07 11:26:51.378 | INFO     | __main__:val1:19 - recall25:0.08732876712328767
2020-06-07 11:26:51.381 | INFO     | __main__:val1:19 - recall50:0.125
2020-06-07 11:26:51.383 | INFO     | __main__:val1:19 - recall75:0.15182648401826485
2020-06-07 11:26:51.385 | INFO     | __main__:val1:19 - recall100:0.17636986301

1


2020-06-07 11:28:36.180 | INFO     | __main__:val1:19 - recall25:0.10072342793544797
2020-06-07 11:28:36.183 | INFO     | __main__:val1:19 - recall50:0.13578185865331108
2020-06-07 11:28:36.185 | INFO     | __main__:val1:19 - recall75:0.1686143572621035
2020-06-07 11:28:36.187 | INFO     | __main__:val1:19 - recall100:0.1930996104618809
2020-06-07 11:29:45.744 | INFO     | __main__:val1:19 - recall25:0.09845984598459846
2020-06-07 11:29:45.747 | INFO     | __main__:val1:19 - recall50:0.13861386138613863
2020-06-07 11:29:45.750 | INFO     | __main__:val1:19 - recall75:0.1639163916391639
2020-06-07 11:29:45.751 | INFO     | __main__:val1:19 - recall100:0.19086908690869087
2020-06-07 11:30:51.708 | INFO     | __main__:val1:19 - recall25:0.0884703196347032
2020-06-07 11:30:51.712 | INFO     | __main__:val1:19 - recall50:0.1267123287671233
2020-06-07 11:30:51.713 | INFO     | __main__:val1:19 - recall75:0.15296803652968036
2020-06-07 11:30:51.715 | INFO     | __main__:val1:19 - recall100:0.

2


...

2020-06-07 11:52:39.843 | INFO     | __main__:val1:19 - recall25:0.09849749582637729
2020-06-07 11:52:39.846 | INFO     | __main__:val1:19 - recall50:0.13244296048970505
2020-06-07 11:52:39.848 | INFO     | __main__:val1:19 - recall75:0.16304952698942682
2020-06-07 11:52:39.850 | INFO     | __main__:val1:19 - recall100:0.18753478018920422
2020-06-07 11:53:49.547 | INFO     | __main__:val1:19 - recall25:0.1001100110011001
2020-06-07 11:53:49.551 | INFO     | __main__:val1:19 - recall50:0.13806380638063806
2020-06-07 11:53:49.554 | INFO     | __main__:val1:19 - recall75:0.16666666666666666
2020-06-07 11:53:49.556 | INFO     | __main__:val1:19 - recall100:0.19636963696369636
2020-06-07 11:54:55.314 | INFO     | __main__:val1:19 - recall25:0.09018264840182648
2020-06-07 11:54:55.318 | INFO     | __main__:val1:19 - recall50:0.1278538812785388
2020-06-07 11:54:55.320 | INFO     | __main__:val1:19 - recall75:0.1552511415525114
2020-06-07 11:54:55.322 | INFO     | __main__:val1:19 - recall100:

8


2020-06-07 11:56:39.365 | INFO     | __main__:val1:19 - recall25:0.09961046188091263
2020-06-07 11:56:39.367 | INFO     | __main__:val1:19 - recall50:0.13244296048970505
2020-06-07 11:56:39.369 | INFO     | __main__:val1:19 - recall75:0.16249304396215916
2020-06-07 11:56:39.371 | INFO     | __main__:val1:19 - recall100:0.18864774624373956
2020-06-07 11:57:49.024 | INFO     | __main__:val1:19 - recall25:0.09955995599559957
2020-06-07 11:57:49.028 | INFO     | __main__:val1:19 - recall50:0.13641364136413642
2020-06-07 11:57:49.030 | INFO     | __main__:val1:19 - recall75:0.16611661166116612
2020-06-07 11:57:49.033 | INFO     | __main__:val1:19 - recall100:0.19471947194719472
2020-06-07 11:58:55.177 | INFO     | __main__:val1:19 - recall25:0.09018264840182648
2020-06-07 11:58:55.180 | INFO     | __main__:val1:19 - recall50:0.12899543378995434
2020-06-07 11:58:55.181 | INFO     | __main__:val1:19 - recall75:0.1558219178082192
2020-06-07 11:58:55.183 | INFO     | __main__:val1:19 - recall10

9


2020-06-07 12:00:39.818 | INFO     | __main__:val1:19 - recall25:0.09794101279910963
2020-06-07 12:00:39.821 | INFO     | __main__:val1:19 - recall50:0.1318864774624374
2020-06-07 12:00:39.824 | INFO     | __main__:val1:19 - recall75:0.16082359488035614
2020-06-07 12:00:39.825 | INFO     | __main__:val1:19 - recall100:0.18530884808013356
2020-06-07 12:01:49.824 | INFO     | __main__:val1:19 - recall25:0.10121012101210121
2020-06-07 12:01:49.828 | INFO     | __main__:val1:19 - recall50:0.13806380638063806
2020-06-07 12:01:49.830 | INFO     | __main__:val1:19 - recall75:0.16886688668866887
2020-06-07 12:01:49.832 | INFO     | __main__:val1:19 - recall100:0.19636963696369636
2020-06-07 12:02:56.125 | INFO     | __main__:val1:19 - recall25:0.091324200913242
2020-06-07 12:02:56.128 | INFO     | __main__:val1:19 - recall50:0.1284246575342466
2020-06-07 12:02:56.129 | INFO     | __main__:val1:19 - recall75:0.15639269406392695
2020-06-07 12:02:56.131 | INFO     | __main__:val1:19 - recall100:0

In [197]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    rp = model.predict_proba(val_data)[:, 1]
    
    dfa.loc[dfa['phase']==7, 'fscore'] = rp[:dfa[dfa['phase']==7].shape[0]]
    dfa.loc[dfa['phase']==7, 'frank'] = dfa[dfa['phase']==7].groupby('user_id')['fscore'].rank(method='first', ascending=False)
    r = val1(dfa[(dfa['phase']==7)&(dfa['frank']<101)], 100, all_click_df[7][1])

    dfa.loc[dfa['phase']==8, 'fscore'] = rp[dfa[dfa['phase']==7].shape[0]:(dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])]
    dfa.loc[dfa['phase']==8, 'frank'] = dfa[dfa['phase']==8].groupby('user_id')['fscore'].rank(method='first', ascending=False)
    r = val1(dfa[(dfa['phase']==8)&(dfa['frank']<101)], 100, all_click_df[8][1])

    ta = (dfa[dfa['phase']==7].shape[0]+dfa[dfa['phase']==8].shape[0])

    dfa.loc[dfa['phase']==9, 'fscore'] = rp[ta:(ta+dfa[dfa['phase']==9].shape[0])]
    dfa.loc[dfa['phase']==9, 'frank'] = dfa[dfa['phase']==9].groupby('user_id')['fscore'].rank(method='first', ascending=False)
    r = val1(dfa[(dfa['phase']==9)&(dfa['frank']<101)], 100, all_click_df[9][1])

#     dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

#     dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

#     r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0


2020-06-07 12:04:40.689 | INFO     | __main__:val1:19 - recall25:0.09738452977184196
2020-06-07 12:04:40.693 | INFO     | __main__:val1:19 - recall50:0.13244296048970505
2020-06-07 12:04:40.695 | INFO     | __main__:val1:19 - recall75:0.16249304396215916
2020-06-07 12:04:40.696 | INFO     | __main__:val1:19 - recall100:0.18586533110740122
2020-06-07 12:05:52.276 | INFO     | __main__:val1:19 - recall25:0.10121012101210121
2020-06-07 12:05:52.280 | INFO     | __main__:val1:19 - recall50:0.13806380638063806
2020-06-07 12:05:52.281 | INFO     | __main__:val1:19 - recall75:0.16666666666666666
2020-06-07 12:05:52.284 | INFO     | __main__:val1:19 - recall100:0.19636963696369636
2020-06-07 12:06:58.787 | INFO     | __main__:val1:19 - recall25:0.09018264840182648
2020-06-07 12:06:58.791 | INFO     | __main__:val1:19 - recall50:0.1295662100456621
2020-06-07 12:06:58.793 | INFO     | __main__:val1:19 - recall75:0.15753424657534246
2020-06-07 12:06:58.794 | INFO     | __main__:val1:19 - recall10

1


2020-06-07 12:08:41.990 | INFO     | __main__:val1:19 - recall25:0.09849749582637729
2020-06-07 12:08:41.993 | INFO     | __main__:val1:19 - recall50:0.1335559265442404
2020-06-07 12:08:41.995 | INFO     | __main__:val1:19 - recall75:0.1652754590984975
2020-06-07 12:08:41.998 | INFO     | __main__:val1:19 - recall100:0.18753478018920422
2020-06-07 12:09:52.137 | INFO     | __main__:val1:19 - recall25:0.10176017601760176
2020-06-07 12:09:52.141 | INFO     | __main__:val1:19 - recall50:0.14026402640264027
2020-06-07 12:09:52.144 | INFO     | __main__:val1:19 - recall75:0.1694169416941694
2020-06-07 12:09:52.146 | INFO     | __main__:val1:19 - recall100:0.19746974697469746
2020-06-07 12:10:58.822 | INFO     | __main__:val1:19 - recall25:0.09075342465753425
2020-06-07 12:10:58.825 | INFO     | __main__:val1:19 - recall50:0.1312785388127854
2020-06-07 12:10:58.828 | INFO     | __main__:val1:19 - recall75:0.1569634703196347
2020-06-07 12:10:58.830 | INFO     | __main__:val1:19 - recall100:0.

2


...

2020-06-07 12:32:51.715 | INFO     | __main__:val1:19 - recall25:0.09627156371730662
2020-06-07 12:32:51.718 | INFO     | __main__:val1:19 - recall50:0.13132999443516974
2020-06-07 12:32:51.721 | INFO     | __main__:val1:19 - recall75:0.16082359488035614
2020-06-07 12:32:51.723 | INFO     | __main__:val1:19 - recall100:0.18308291597106288
2020-06-07 12:34:01.962 | INFO     | __main__:val1:19 - recall25:0.10231023102310231
2020-06-07 12:34:01.966 | INFO     | __main__:val1:19 - recall50:0.14026402640264027
2020-06-07 12:34:01.968 | INFO     | __main__:val1:19 - recall75:0.17106710671067107
2020-06-07 12:34:01.970 | INFO     | __main__:val1:19 - recall100:0.19691969196919692
2020-06-07 12:35:08.501 | INFO     | __main__:val1:19 - recall25:0.091324200913242
2020-06-07 12:35:08.504 | INFO     | __main__:val1:19 - recall50:0.1295662100456621
2020-06-07 12:35:08.506 | INFO     | __main__:val1:19 - recall75:0.1558219178082192
2020-06-07 12:35:08.508 | INFO     | __main__:val1:19 - recall100:0

8


2020-06-07 12:36:54.296 | INFO     | __main__:val1:19 - recall25:0.09571508069003895
2020-06-07 12:36:54.300 | INFO     | __main__:val1:19 - recall50:0.13132999443516974
2020-06-07 12:36:54.302 | INFO     | __main__:val1:19 - recall75:0.16138007790762382
2020-06-07 12:36:54.304 | INFO     | __main__:val1:19 - recall100:0.18308291597106288
2020-06-07 12:38:05.451 | INFO     | __main__:val1:19 - recall25:0.10121012101210121
2020-06-07 12:38:05.455 | INFO     | __main__:val1:19 - recall50:0.1408140814081408
2020-06-07 12:38:05.457 | INFO     | __main__:val1:19 - recall75:0.1705170517051705
2020-06-07 12:38:05.459 | INFO     | __main__:val1:19 - recall100:0.19691969196919692
2020-06-07 12:39:12.184 | INFO     | __main__:val1:19 - recall25:0.09189497716894977
2020-06-07 12:39:12.187 | INFO     | __main__:val1:19 - recall50:0.1312785388127854
2020-06-07 12:39:12.190 | INFO     | __main__:val1:19 - recall75:0.15810502283105024
2020-06-07 12:39:12.192 | INFO     | __main__:val1:19 - recall100:

9


2020-06-07 12:40:57.136 | INFO     | __main__:val1:19 - recall25:0.09571508069003895
2020-06-07 12:40:57.139 | INFO     | __main__:val1:19 - recall50:0.13132999443516974
2020-06-07 12:40:57.141 | INFO     | __main__:val1:19 - recall75:0.16193656093489148
2020-06-07 12:40:57.143 | INFO     | __main__:val1:19 - recall100:0.18308291597106288
2020-06-07 12:42:07.933 | INFO     | __main__:val1:19 - recall25:0.10176017601760176
2020-06-07 12:42:07.936 | INFO     | __main__:val1:19 - recall50:0.1408140814081408
2020-06-07 12:42:07.938 | INFO     | __main__:val1:19 - recall75:0.1705170517051705
2020-06-07 12:42:07.940 | INFO     | __main__:val1:19 - recall100:0.19746974697469746
2020-06-07 12:43:14.659 | INFO     | __main__:val1:19 - recall25:0.0930365296803653
2020-06-07 12:43:14.662 | INFO     | __main__:val1:19 - recall50:0.1324200913242009
2020-06-07 12:43:14.665 | INFO     | __main__:val1:19 - recall75:0.15753424657534246
2020-06-07 12:43:14.667 | INFO     | __main__:val1:19 - recall100:0

In [83]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    dff.loc[dff['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    dff.loc[dff['phase']==6, 'frank'] = dff.loc[dff['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(dff.loc[(dff['phase']==6)&(dff['frank']<101)], 100, all_click_df[6][1])

0


2020-06-05 15:52:11.448 | INFO     | __main__:val1:19 - recall25:0.0757825370675453
2020-06-05 15:52:11.452 | INFO     | __main__:val1:19 - recall50:0.1587040087863811
2020-06-05 15:52:11.454 | INFO     | __main__:val1:19 - recall75:0.20373421197144426
2020-06-05 15:52:11.456 | INFO     | __main__:val1:19 - recall100:0.2300933552992861


1


2020-06-05 15:53:51.033 | INFO     | __main__:val1:19 - recall25:0.07633168588687535
2020-06-05 15:53:51.036 | INFO     | __main__:val1:19 - recall50:0.15815485996705106
2020-06-05 15:53:51.038 | INFO     | __main__:val1:19 - recall75:0.20373421197144426
2020-06-05 15:53:51.040 | INFO     | __main__:val1:19 - recall100:0.2300933552992861


2


...

2020-06-05 16:03:52.749 | INFO     | __main__:val1:19 - recall25:0.0757825370675453
2020-06-05 16:03:52.752 | INFO     | __main__:val1:19 - recall50:0.16035145524437122
2020-06-05 16:03:52.754 | INFO     | __main__:val1:19 - recall75:0.2015376166941241
2020-06-05 16:03:52.755 | INFO     | __main__:val1:19 - recall100:0.22679846238330587


8


2020-06-05 16:05:32.421 | INFO     | __main__:val1:19 - recall25:0.07742998352553541
2020-06-05 16:05:32.424 | INFO     | __main__:val1:19 - recall50:0.16035145524437122
2020-06-05 16:05:32.425 | INFO     | __main__:val1:19 - recall75:0.20208676551345414
2020-06-05 16:05:32.427 | INFO     | __main__:val1:19 - recall100:0.2257001647446458


9


KeyboardInterrupt: 

In [110]:
model.save_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500_15_6.dump'))

# predict

In [9]:
item_feat = get_item_data()

item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

item_feat.set_index('item_id', inplace=True)

/root/data1/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [94]:
r_itemcf_yl = []
r_itemcf = []
r_binn = []
r_list_w2v = []

# r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_test_0527.pkl'))

# r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_test_0527.pkl'))

# r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_test_0527.pkl'))
# r_list_w2v = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_w2v_test_0603_all.pkl'))

r_itemcf_yl.extend(PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_test_0605_789.pkl')))
r_itemcf.extend(PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_test_0605_789.pkl')))
r_binn.extend(PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_test_0605_789.pkl')))
r_list_w2v.extend(PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_w2v_test_0605_789.pkl')))

In [95]:
# phase = [0, 1, 2, 3, 4, 5, 6]
phase = [7, 8, 9]
match_num = 500

df = data_preporcess([r_itemcf, r_binn, r_itemcf_yl, r_list_w2v], match_num, phase, mode='predict')


merge Multi-channel recall...


100%|██████████| 3/3 [03:14<00:00, 64.91s/it]


In [133]:
df['t'] = range(df.shape[0])

In [124]:
df.head(2)

,user_id,item_id_pred,score_x,rank_x,phase,score_y,rank_y,score_x,rank_x,score_y,rank_y,item_cnt,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec
0,11,10528,9.343545,1.0,0,0.749194,21.0,0.110274,2.0,34.100440,4.0,7.0,28210,64689,20317,"[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3..."
1,11,79868,6.949118,2.0,0,2.023002,1.0,0.007360,322.0,10.992932,454.0,47.0,28210,64689,20317,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48..."


In [128]:
temp_ = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features.pkl'))

In [129]:
temp_.columns

Index(['user_id', 'item_id_pred', 'phase', 'score_x', 'rank_x', 'score_y',
       'rank_y', 'score', 'rank', 'item_cnt', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img', 'sim4_text',
       'sim4_img', 'sim5_text', 'sim5_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim4_text_img', 'sim5_text_img',
       'sim12_text', 'sim123_text', 'sim1234_text', 'sim12345_text',
       'sim12_img', 'sim123_img', 'sim1234_img', 'sim12345_img',
       'sim12_text_img', 'sim123_text_img', 'sim1234_text_img',
       'sim12345_text_img'],
      dtype='object

In [134]:
df_ = pd.merge(df, temp_[['user_id', 'item_id_pred', 'phase', 'sim1_text', 'sim1_img',
       'sim2_text', 'sim2_img', 'sim3_text', 'sim3_img']], on=['user_id', 'item_id_pred', 'phase'])

In [132]:
df_.shape

(11168101, 23)

In [135]:
df1 = df[~df.t.isin(df_.t)]

In [136]:
df1.shape

(4362053, 18)

In [137]:
df1.head(2)

,user_id,item_id_pred,score_x,rank_x,phase,score_y,rank_y,score_x,rank_x,score_y,rank_y,item_cnt,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,t
875,11,81221,0.0,0.0,0,0.0,0.0,0.0,0.0,39.036631,2.0,5.0,28210,64689,20317,"[2.7964396476745605, -0.6318608522415161, 2.20...","[-1.5980700254440308, 2.5623795986175537, 1.70...",875
876,11,11679,0.0,0.0,0,0.0,0.0,0.0,0.0,29.522777,7.0,4.0,28210,64689,20317,"[-0.25924938917160034, 1.6548818349838257, 0.5...","[0.8699896335601807, -4.748091220855713, -0.07...",876


In [98]:
df.columns = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3']

In [99]:
df['item_id_pred_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

df['item_id_pred_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

for c in tqdm('last_1 last_2 last_3'.split()):
    df[f'{c}_text_vec'] = df[c].map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df[f'{c}_img_vec'] = df[c].map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

logger.info('sim1_text')
df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last_1_text_vec']).map(sum)
logger.info('sim1_text')
df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last_1_img_vec']).map(sum)
logger.info('sim1_text')
df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last_2_text_vec']).map(sum)
logger.info('sim1_text')
df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last_2_img_vec']).map(sum)
logger.info('sim1_text')
df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last_3_text_vec']).map(sum)
logger.info('sim1_text')
df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last_3_img_vec']).map(sum)

100%|██████████| 3/3 [12:40<00:00, 253.63s/it]
2020-06-06 09:57:24.533 | INFO     | __main__:<module>:10 - sim1_text
2020-06-06 10:01:24.432 | INFO     | __main__:<module>:12 - sim1_text
2020-06-06 10:05:29.765 | INFO     | __main__:<module>:14 - sim1_text
2020-06-06 10:09:31.669 | INFO     | __main__:<module>:16 - sim1_text
2020-06-06 10:13:41.242 | INFO     | __main__:<module>:18 - sim1_text
2020-06-06 10:17:37.850 | INFO     | __main__:<module>:20 - sim1_text


In [105]:
df.head(2)

,user_id,item_id_pred,score_1,rank_1,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,item_cnt,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank1_sub_rank4,rank1_add_rank4,rank1_mul_rank4,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,rank2_sub_rank4,rank2_add_rank4,rank2_mul_rank4,rank3_sub_rank4,rank3_add_rank4,rank3_mul_rank4,rank1_add_rank2_add_rank3,rank1_mul_rank2_mul_rank3,rank1_add_rank2_add_rank4,rank1_mul_rank2_mul_rank4,rank4_add_rank2_add_rank3,rank4_mul_rank2_mul_rank3,rank1_add_rank2_add_rank3_add_rank4,rank1_mul_rank2_mul_rank3_mul_rank4,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score1_sub_score4,score1_add_score4,score1_mul_score4,score2_sub_score3,score2_add_score3,score2_mul_score3,score2_sub_score4,score2_add_score4,score2_mul_score4,score3_sub_score4,score3_add_score4,score3_mul_score4,score1_add_score2_add_score3,score1_mul_score2_mul_score3,score1_add_score2_add_score4,score1_mul_score2_mul_score4,score4_add_score2_add_score3,score4_mul_score2_mul_score3,score1_add_score2_add_score3_add_score4,score1_mul_score2_mul_score3_mul_score4,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img
0,7,79771,92.351740,1.0,7,15.879384,3.0,0.296926,1.0,144.723286,2.0,115.0,44390,80626,12845,"[1.4303220510482788, -1.7596940994262695, -1.1...","[-2.18737268447876, -0.2341355383396149, -1.08...","[0.8865178823471069, 1.5853179693222046, 0.872...","[-1.3981404304504397, 0.4708309173583984, -0.6...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",-40.597596,301.261393,0.0,1271.163407,0.0,1271.163407,-2.0,4.0,3.0,0.0,2.0,1.0,-1.0,3.0,2.0,2.0,4.0,3.0,1.0,5.0,6.0,-1.0,3.0,2.0,5.0,3.0,6.0,6.0,6.0,6.0,7.0,6.0,76.472356,108.231124,1466.488720,92.054814,92.648666,27.421649,-52.371546,237.075026,13365.447314,15.582458,16.176310,4.715005,-128.843902,160.602670,2298.116594,-144.42636,145.020212,42.972132,108.528050,435.438889,252.954410,212235.066592,160.899596,682.370974,253.251336,63018.146936,-12230.48828,0.0,0.0,-40.597596,-40.597596,1572.424800,2.843588e+03,-12230.48828,-12230.48828
1,7,113031,75.163632,2.0,7,15.739498,4.0,0.266439,2.0,130.546430,14.0,174.0,44390,80626,12845,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.8865178823471069, 1.5853179693222046, 0.872...","[-1.3981404304504397, 0.4708309173583984, -0.6...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0.000000,175.514567,0.0,690880.000000,0.0,690880.000000,-2.0,6.0,8.0,0.0,4.0,4.0,-12.0,16.0,28.0,2.0,6.0,8.0,-10.0,18.0,56.0,-12.0,16.0,28.0,8.0,16.0,20.0,112.0,20.0,112.0,22.0,224.0,59.424133,90.903130,1183.037831,74.897192,75.430071,20.026537,-55.382798,205.710061,9812.343727,15.473059,16.005937,4.193619,-114.806932,146.285928,2054.735272,-130.27999,130.812869,34.782684,91.169569,315.207636,221.449559,154441.364855,146.552367,547.461993,221.715998,41149.231499,0.00000,0.0,0.0,0.000000,0.000000,691055.514567,1.381936e+06,0.00000,0.00000


In [104]:
df = fe(df)

100%|██████████| 2/2 [00:00<00:00,  2.59it/s]


In [103]:
df.shape

(6604213, 29)

In [147]:
dff.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'item_id_pred_text_vec',
       'item_id_pred_img_vec', 't', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_su

In [106]:
cols = ['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', 'score3_sub_score4',
       'score3_add_score4', 'score3_mul_score4', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img',
       'rank1_add_rank2_add_rank3', 'rank1_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank4', 'rank1_mul_rank2_mul_rank4',
       'rank4_add_rank2_add_rank3', 'rank4_mul_rank2_mul_rank3',
       'rank1_add_rank2_add_rank3_add_rank4',
       'rank1_mul_rank2_mul_rank3_mul_rank4', 'score1_add_score2_add_score3',
       'score1_mul_score2_mul_score3', 'score1_add_score2_add_score4',
       'score1_mul_score2_mul_score4', 'score4_add_score2_add_score3',
       'score4_mul_score2_mul_score3',
       'score1_add_score2_add_score3_add_score4',
       'score1_mul_score2_mul_score3_mul_score4']
# df[cols].to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features_789.pkl'))

In [7]:
df = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{500}_test_features_789.pkl'))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/contest/kdd_debiasing/data_gen/dft500_test_features_789.pkl'

## m

In [111]:
dffp = df[cols]

In [47]:
dffp = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}_test_features_all_new.pkl'))

In [112]:
dffp.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', '

In [49]:
def t1(s):
    return dffp.item_id_pred.map(lambda x: item_feat[f'{s}_vec'][x] if x in item_feat[f'{s}_vec'] else np.zeros(128))

In [50]:
pool = Pool(2)
r1 = pool.map(t1, 'text img'.split())

In [51]:
dffp['item_id_pred_text_vec'] = r1[0]
dffp['item_id_pred_img_vec'] = r1[1]

In [52]:


def t2(c):
#     dff[f'{c}_text_vec']

    c, a = c.split(',')
    print(c, a)
    return dffp[c].map(lambda x: item_feat[f'{a}_vec'][x] if x in item_feat[f'{a}_vec'] else np.zeros(128))


In [53]:
from multiprocessing.dummy import Pool
pool = Pool(6)

r = pool.map(t2, 'last_1,text last_2,text last_3,text last_1,img last_2,img last_3,img'.split())

last_1last_2 text
last_3last_1 img
last_2 img
last_3 text  
img
text


In [54]:
dffp['last1_text_vec'] = r[0]

dffp['last1_img_vec'] = r[3]

dffp['last2_text_vec'] = r[1]
dffp['last2_img_vec'] = r[4]

dffp['last3_text_vec'] = r[2]
dffp['last3_img_vec'] = r[5]

In [55]:
logger.info('sim1_text')
dffp['sim1_text'] = (dffp['item_id_pred_text_vec'] * dffp['last1_text_vec']).map(sum)
logger.info('sim1_img')
dffp['sim1_img'] = (dffp['item_id_pred_img_vec'] * dffp['last1_img_vec']).map(sum)
logger.info('sim2_text')
dffp['sim2_text'] = (dffp['item_id_pred_text_vec'] * dffp['last2_text_vec']).map(sum)
logger.info('sim2_img')
dffp['sim2_img'] = (dffp['item_id_pred_img_vec'] * dffp['last2_img_vec']).map(sum)
logger.info('sim3_text')
dffp['sim3_text'] = (dffp['item_id_pred_text_vec'] * dffp['last3_text_vec']).map(sum)
logger.info('sim3_img')
dffp['sim3_img'] = (dffp['item_id_pred_img_vec'] * dffp['last3_img_vec']).map(sum)

2020-06-05 14:01:47.933 | INFO     | __main__:<module>:1 - sim1_text
2020-06-05 14:12:26.530 | INFO     | __main__:<module>:3 - sim1_img
2020-06-05 14:22:37.708 | INFO     | __main__:<module>:5 - sim2_text
2020-06-05 14:33:24.569 | INFO     | __main__:<module>:7 - sim2_img
2020-06-05 14:43:51.358 | INFO     | __main__:<module>:9 - sim3_text
2020-06-05 14:53:22.057 | INFO     | __main__:<module>:11 - sim3_img


In [63]:
dffp.head(2)

,user_id,item_id_pred,score_1,rank_1,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,item_cnt,last_1,last_2,last_3,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank1_sub_rank4,rank1_add_rank4,rank1_mul_rank4,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,rank2_sub_rank4,rank2_add_rank4,rank2_mul_rank4,rank3_sub_rank4,rank3_add_rank4,rank3_mul_rank4,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score1_sub_score4,score1_add_score4,score1_mul_score4,score2_sub_score3,score2_add_score3,score2_mul_score3,score2_sub_score4,score2_add_score4,score2_mul_score4,score3_sub_score4,score3_add_score4,score3_mul_score4,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img,rank1_add_rank2_add_rank3,rank1_mul_rank2_mul_rank3,rank1_add_rank2_add_rank4,rank1_mul_rank2_mul_rank4,rank4_add_rank2_add_rank3,rank4_mul_rank2_mul_rank3,rank1_add_rank2_add_rank3_add_rank4,rank1_mul_rank2_mul_rank3_mul_rank4,score1_add_score2_add_score3,score1_mul_score2_mul_score3,score1_add_score2_add_score4,score1_mul_score2_mul_score4,score4_add_score2_add_score3,score4_mul_score2_mul_score3,score1_add_score2_add_score3_add_score4,score1_mul_score2_mul_score3_mul_score4,item_id_pred_text_vec,item_id_pred_img_vec,last1_text_vec,last1_img_vec,last2_text_vec,last2_img_vec,last3_text_vec,last3_img_vec
875,11,81221,0.0,0.0,0,0.0,0.0,0.0,0.0,39.036631,2.0,5.0,28210,64689,20317,412.371871,165.867255,239.330954,61.103960,400.772712,172.954381,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0,-2.0,2.0,0.0,-2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-39.036631,39.036631,0.0,0.0,0.0,0.0,-39.036631,39.036631,0.0,-39.036631,39.036631,0.0,68398.990458,14624.068977,69315.396136,651.702826,1052.475538,226.971215,399.925595,83023.059436,152338.455572,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,39.036631,0.0,39.036631,0.0,39.036631,0.0,"[2.7964396476745605, -0.6318608522415161, 2.20...","[-1.5980700254440308, 2.5623795986175537, 1.70...","[6.135307788848878, -2.298557758331299, 3.1121...","[-3.51704216003418, 0.7559890151023865, 1.1019...","[-0.4763919115066528, 2.0025746822357178, -1.5...","[-0.02582862228155136, -4.5208396911621085, -2...","[0.9810895323753356, 2.146204233169556, -1.869...","[-1.991158962249756, 0.3152694702148437, -5.20..."
876,11,11679,0.0,0.0,0,0.0,0.0,0.0,0.0,29.522777,7.0,4.0,28210,64689,20317,24.700372,27.944465,31.395093,238.904828,197.612647,109.214818,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,7.0,0.0,0.0,0.0,0.0,-7.0,7.0,0.0,-7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-29.522777,29.522777,0.0,0.0,0.0,0.0,-29.522777,29.522777,0.0,-29.522777,29.522777,0.0,690.238693,7500.439234,21582.229371,56.095465,253.708112,266.849294,376.064112,8190.677927,29772.907298,0.0,0.0,7.0,0.0,7.0,0.0,7.0,0.0,0.0,0.0,29.522777,0.0,29.522777,0.0,29.522777,0.0,"[-0.25924938917160034, 1.6548818349838257, 0.5...","[0.8699896335601807, -4.748091220855713, -0.07...","[6.135307788848878, -2.298557758331299, 3.1121...","[-3.51704216003418, 0.7559890151023865, 1.1019...","[-0.4763919115066528, 2.0025746822357178, -1.5...","[-0.02582862228155136, -4.5208396911621085, -2...","[0.9810895323753356, 2.146204233169556, -1.869...","[-1.991158962249756, 0.3152694702148437, -5.20..."


In [58]:
dffp.columns

Index(['user_id', 'item_id_pred', 'score_1', 'rank_1', 'phase', 'score_2',
       'rank_2', 'score_3', 'rank_3', 'score_4', 'rank_4', 'item_cnt',
       'last_1', 'last_2', 'last_3', 'sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'rank1_sub_rank2',
       'rank1_add_rank2', 'rank1_mul_rank2', 'rank1_sub_rank3',
       'rank1_add_rank3', 'rank1_mul_rank3', 'rank1_sub_rank4',
       'rank1_add_rank4', 'rank1_mul_rank4', 'rank2_sub_rank3',
       'rank2_add_rank3', 'rank2_mul_rank3', 'rank2_sub_rank4',
       'rank2_add_rank4', 'rank2_mul_rank4', 'rank3_sub_rank4',
       'rank3_add_rank4', 'rank3_mul_rank4', 'score1_sub_score2',
       'score1_add_score2', 'score1_mul_score2', 'score1_sub_score3',
       'score1_add_score3', 'score1_mul_score3', 'score1_sub_score4',
       'score1_add_score4', 'score1_mul_score4', 'score2_sub_score3',
       'score2_add_score3', 'score2_mul_score3', 'score2_sub_score4',
       'score2_add_score4', 'score2_mul_score4', '

In [61]:
dffp = fe(dffp)

In [62]:
dffp[['sim1_text', 'sim1_img', 'sim2_text',
       'sim2_img', 'sim3_text', 'sim3_img', 'sim1_text_img',
       'sim2_text_img', 'sim3_text_img', 'sim12_text', 'sim123_text',
       'sim12_img', 'sim123_img', 'sim12_text_img', 'sim123_text_img']].head(5)

,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img
875,412.371871,165.867255,239.330954,61.103960,400.772712,172.954381,68398.990458,14624.068977,69315.396136,651.702826,1052.475538,226.971215,399.925595,83023.059436,152338.455572
876,24.700372,27.944465,31.395093,238.904828,197.612647,109.214818,690.238693,7500.439234,21582.229371,56.095465,253.708112,266.849294,376.064112,8190.677927,29772.907298
877,344.369137,130.674165,119.010146,57.891801,97.820684,56.114521,45000.149376,6889.711704,5489.160804,463.379282,561.199966,188.565966,244.680487,51889.861080,57379.021884
878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
879,201.662093,136.008802,274.297705,133.907236,172.537462,116.664095,27427.819637,36730.447457,20128.926862,475.959798,648.497260,269.916038,386.580133,64158.267094,84287.193956


In [113]:
test_data, cat_cols = get_model_input(dffp, mode='predict')


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
# set(df.user_id)-set(dft.user_id) # {'14997'}

{'14997'}

In [177]:
# from catboost import Pool, CatBoostClassifier

# model = CatBoostClassifier()
# model.load_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500_feat_last123_all.dump'))

In [205]:
from catboost import Pool, CatBoostClassifier

model7 = CatBoostClassifier()
model7.load_model(os.path.join(args.DATA_DIR, f'data_gen/catboost_model500_16_7.dump'))

model8 = CatBoostClassifier()
model8.load_model(os.path.join(args.DATA_DIR, f'data_gen/catboost_model500_16_8.dump'))

model9 = CatBoostClassifier()
model9.load_model(os.path.join(args.DATA_DIR, f'data_gen/catboost_model500_16_9.dump'))

In [207]:
dffp.loc[dffp.phase==7, 'fscore'] = model7.predict_proba(test_data)[:, 1][:dffp.loc[dffp.phase==7].shape[0]]

/root/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [209]:
ta = dffp.loc[dffp.phase==7].shape[0]
dffp.loc[dffp.phase==8, 'fscore'] = model8.predict_proba(test_data)[:, 1][ta:(ta+dffp.loc[dffp.phase==8].shape[0])]

In [210]:
ta = dffp.loc[dffp.phase==7].shape[0] + dffp.loc[dffp.phase==8].shape[0]
dffp.loc[dffp.phase==9, 'fscore'] = model9.predict_proba(test_data)[:, 1][ta:(ta+dffp.loc[dffp.phase==9].shape[0])]

In [211]:
dffp.loc[:, 'frank'] = dffp.groupby('user_id')['fscore'].rank(method='first', ascending=False)

In [212]:
dffp.sort_values('user_id frank'.split()).head(20)

,user_id,item_id_pred,score_1,rank_1,phase,score_2,rank_2,score_3,rank_3,score_4,rank_4,item_cnt,last_1,last_2,last_3,sim1_text,sim1_img,sim2_text,sim2_img,sim3_text,sim3_img,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank1_sub_rank4,rank1_add_rank4,rank1_mul_rank4,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,rank2_sub_rank4,rank2_add_rank4,rank2_mul_rank4,rank3_sub_rank4,rank3_add_rank4,rank3_mul_rank4,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score1_sub_score4,score1_add_score4,score1_mul_score4,score2_sub_score3,score2_add_score3,score2_mul_score3,score2_sub_score4,score2_add_score4,score2_mul_score4,score3_sub_score4,score3_add_score4,score3_mul_score4,sim1_text_img,sim2_text_img,sim3_text_img,sim12_text,sim123_text,sim12_img,sim123_img,sim12_text_img,sim123_text_img,rank1_add_rank2_add_rank3,rank1_mul_rank2_mul_rank3,rank1_add_rank2_add_rank4,rank1_mul_rank2_mul_rank4,rank4_add_rank2_add_rank3,rank4_mul_rank2_mul_rank3,rank1_add_rank2_add_rank3_add_rank4,rank1_mul_rank2_mul_rank3_mul_rank4,score1_add_score2_add_score3,score1_mul_score2_mul_score3,score1_add_score2_add_score4,score1_mul_score2_mul_score4,score4_add_score2_add_score3,score4_mul_score2_mul_score3,score1_add_score2_add_score3_add_score4,score1_mul_score2_mul_score3_mul_score4,fscore,frank
2795517,10007,82600,2.990922,142.0,8,0.000000,0.0,0.017854,127.0,0.000000,0.0,3.0,18811,26711,26047,553.589627,309.363419,0.0,2706.142908,380.164830,113.603444,142.0,142.0,0.0,15.0,269.0,18034.0,142.0,142.0,0.0,-127.0,127.0,0.0,0.0,0.0,0.0,127.0,127.0,0.0,2.990922,2.990922,0.000000,2.973068,3.008777,0.053401,2.990922,2.990922,0.000000,-0.017854,0.017854,0.000000,0.000000,0.000000,0.000000,0.017854,0.017854,0.000000,171260.379446,0.0,43188.033960,553.589627,933.754457,3015.506326,3129.109770,171260.379446,214448.413406,269.0,0.0,142.0,0.0,127.0,0.0,269.0,0.0,3.008777,0.000000,2.990922,0.000000,0.017854,0.000000,3.008777,0.00000,0.108638,1.0
2796502,10007,69318,0.000000,0.0,8,0.000000,0.0,0.000000,0.0,27.170245,286.0,3.0,18811,26711,26047,557.689336,345.347969,0.0,2965.548698,584.164960,239.226787,0.0,0.0,0.0,0.0,0.0,0.0,-286.0,286.0,0.0,0.0,0.0,0.0,-286.0,286.0,0.0,-286.0,286.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-27.170245,27.170245,0.000000,0.000000,0.000000,0.000000,-27.170245,27.170245,0.000000,-27.170245,27.170245,0.000000,192596.879623,0.0,139747.906638,557.689336,1141.854295,3310.896667,3550.123455,192596.879623,332344.786261,0.0,0.0,286.0,0.0,286.0,0.0,286.0,0.0,0.000000,0.000000,27.170245,0.000000,27.170245,0.000000,27.170245,0.00000,0.073692,2.0
2795377,10007,20225,12.180432,2.0,8,0.353868,24.0,0.093008,6.0,0.000000,0.0,3.0,18811,26711,26047,376.614733,105.096332,0.0,2458.718829,166.210164,42.067598,-22.0,26.0,48.0,-4.0,8.0,12.0,2.0,2.0,0.0,18.0,30.0,144.0,24.0,24.0,0.0,6.0,6.0,0.0,11.826564,12.534300,4.310263,12.087424,12.273439,1.132872,12.180432,12.180432,0.000000,0.260860,0.446875,0.032912,0.353868,0.353868,0.000000,0.093008,0.093008,0.000000,39580.826932,0.0,6992.062387,376.614733,542.824897,2563.815161,2605.882759,39580.826932,46572.889318,32.0,288.0,26.0,0.0,30.0,0.0,32.0,0.0,12.627307,0.400887,12.534300,0.000000,0.446875,0.000000,12.627307,0.00000,0.067311,3.0
2795495,10007,29710,3.165045,120.0,8,0.000000,0.0,0.004896,362.0,0.000000,0.0,3.0,18811,26711,26047,728.006360,320.716155,0.0,1323.390277,245.906833,263.666880,120.0,120.0,0.0,-242.0,482.0,43440.0,120.0,120.0,0.0,-362.0,362.0,0.0,0.0,0.0,0.0,362.0,362.0,0.0,3.165045,3.165045,0.000000,3.160149,3.169941,0.015495,3.165045,3.165045,0.000000,-0.004896,0.004896,0.000000,0.000000,0.000000,0.000000,0.004896,0.004896,0.000000,233483.400422,0.0,64837.487373,728.006360,973.913193,1644.106432,1907.773311,233483.400422,298320.887794,482.0,0.0,120.0,0.0,362.0,0.0,482.0,0.0,3.169941,0.000000,3.165045,0.000000,0.004896,0.000000,3.169941,0.00000,0.062613,4.0
2796567,10007,33462

In [213]:
df1 = dffp.sort_values(['user_id', 'fscore'], ascending=False)

In [214]:
sub = df1[df1.frank<51]

In [215]:
sub = sub.groupby('user_id')['item_id_pred'].agg(list)

In [121]:
sub

user_id
10007    [997, 66212, 22925, 33462, 75404, 8052, 66873,...
10008    [26642, 14746, 20792, 73662, 30244, 78701, 999...
10017    [40603, 15617, 73336, 109739, 49948, 51104, 11...
10019    [94299, 5609, 102581, 102272, 95979, 75157, 31...
10051    [18389, 21687, 47923, 19330, 76403, 27152, 657...
                               ...                        
998      [79120, 56833, 65163, 23368, 36951, 1077, 3530...
9985     [81407, 116841, 28606, 28, 110688, 47399, 5159...
9986     [56664, 73367, 63892, 29759, 51661, 75381, 411...
9995     [97058, 29863, 14803, 90370, 43525, 48286, 600...
9997     [31696, 78785, 100876, 111307, 59273, 94258, 7...
Name: item_id_pred, Length: 5367, dtype: object

In [216]:
sub

user_id
10007    [82600, 69318, 20225, 29710, 33462, 997, 8052,...
10008    [26642, 14746, 101637, 73662, 66359, 20792, 92...
10017    [40603, 49948, 15617, 109739, 73336, 51104, 16...
10019    [94299, 102272, 111196, 104807, 75157, 95979, ...
10051    [76403, 21687, 47923, 47609, 14946, 112868, 19...
                               ...                        
998      [79120, 65163, 23368, 36951, 56833, 35309, 676...
9985     [116841, 47399, 28606, 81407, 86763, 28, 2611,...
9986     [73367, 51661, 56664, 29759, 63892, 74475, 102...
9995     [97058, 14803, 43428, 107517, 96992, 90370, 45...
9997     [31696, 1437, 116561, 3094, 100876, 59273, 106...
Name: item_id_pred, Length: 5367, dtype: object

In [217]:
sub1 = pd.DataFrame()

for i in range(50):
    sub1[f'c{i}'] = sub.str.get(i)

sub1.applymap(int).to_csv('baseline1_itemcf86666.csv', header=False)

In [218]:
sub1.shape

(5367, 50)